In [1]:
from pipeline import Pipeline, build_csv
import string
import json
import io
import csv
from datetime import datetime
from stop_words import stop_words

pipeline = Pipeline()

In [2]:
@pipeline.task()
def file_to_json():
    with open('hn_stories_2014.json', 'r') as f:
        data = json.load(f)
        stories = data['stories']
    return stories

In [3]:
@pipeline.task(depends_on=file_to_json)
def filter_stories(stories):
    def is_popular(story):
        return (story['points'] > 50) and (story['num_comments'] > 1) and not (story['title'].startswith('Ask HN'))
    
    return (story for story in stories if is_popular(story))

In [4]:
@pipeline.task(depends_on=filter_stories)
def json_to_csv(stories):
    lines = []
    for story in stories:
        lines.append((story['objectID'],
                      datetime.strptime(
                          story['created_at'],
                          "%Y-%m-%dT%H:%M:%SZ"),
                      story['url'],
                      story['points'],
                      story['title']))
        
    return build_csv(lines, header=['objectID', 'created_at', 'url', 'points', 'title'], file=io.StringIO())

In [5]:
@pipeline.task(depends_on=json_to_csv)
def extract_titles(csv_file):
    reader = csv.reader(csv_file)
    header = next(reader)
    idx = header.index('title')
    return (line[idx] for line in reader)

In [6]:
@pipeline.task(depends_on=extract_titles)
def clean_titles(titles):
    for title in titles:
        title = title.lower()
        title = ''.join(c for c in title if c not in string.punctuation)
        yield title

In [7]:
@pipeline.task(depends_on=clean_titles)
def build_keyword_dictionary(titles):
    word_freq = {}
    for title in titles:
        for word in title.split(' '):
            if word and word not in stop_words:
                if word not in word_freq:
                    word_freq[word] = 1
                word_freq[word] += 1
    return word_freq

In [8]:
@pipeline.task(depends_on=build_keyword_dictionary)
def top_words(word_freq):
    freq_tuple = [
        (word, word_freq[word])
        for word in sorted(word_freq, key=word_freq.get, reverse=True)
    ]
    return freq_tuple[:100]

In [9]:
run = pipeline.run()
print(run[top_words])

[('new', 186), ('google', 168), ('bitcoin', 102), ('open', 93), ('programming', 91), ('web', 89), ('data', 86), ('video', 80), ('python', 76), ('code', 73), ('facebook', 72), ('released', 72), ('using', 71), ('2013', 66), ('javascript', 66), ('free', 65), ('source', 65), ('game', 64), ('internet', 63), ('c', 60), ('microsoft', 60), ('linux', 59), ('app', 58), ('pdf', 56), ('language', 55), ('work', 55), ('software', 53), ('2014', 53), ('startup', 52), ('apple', 51), ('make', 51), ('use', 51), ('yc', 49), ('time', 49), ('security', 49), ('github', 46), ('nsa', 46), ('windows', 45), ('way', 42), ('world', 42), ('like', 42), ('computer', 41), ('project', 41), ('1', 41), ('heartbleed', 41), ('ios', 38), ('git', 38), ('dont', 38), ('users', 38), ('design', 38), ('twitter', 37), ('vs', 37), ('developer', 37), ('os', 37), ('ceo', 37), ('life', 37), ('day', 36), ('big', 36), ('online', 35), ('android', 35), ('years', 34), ('court', 34), ('simple', 34), ('apps', 33), ('says', 33), ('learning', 